# Malaysia Job Market Analysis 2024
## Part 2: SQL Analysis

**Goal:** Use SQL CTEs and window functions to rank industries and locations by salary and demand  
**Tools:** Python, SQLite, Pandas  
**Author:** Priyanshu Singh  
**GitHub:** [malaysia-job-market-analysis](https://github.com/maanajipriyanshu/malaysia-job-market-analysis)

## 1. Import Libraries & Load Data
Loading the cleaned dataset and setting up SQLite database for SQL-based analysis.

In [4]:
import pandas as pd
import sqlite3

# load cleaned data
df = pd.read_csv('../data/cleaned/jobstreet_cleaned.csv')

# Creating SQL database
conn = sqlite3.connect('jobstreet.db')
df.to_sql('jobs', conn, if_exists='replace', index=False)

print("Database created successfully!")
result = pd.read_sql("SELECT COUNT(*) as total_jobs FROM jobs", conn)
print(result)

Database created successfully!
   total_jobs
0       69024


## 2. Query 1 :- Salary & Demand Ranking by Industry
Using a **CTE** to calculate industry-level salary stats, then applying **RANK() window function** 
to rank all 26 industries by both average salary and job count simultaneously.

This query answers: Which industries pay the most AND hire the most in Malaysia?

In [5]:
query1 = """
WITH category_stats AS (
    SELECT 
        category,
        COUNT(*) as total_jobs,
        ROUND(AVG(salary_avg), 0) as avg_salary,
        ROUND(MIN(salary_min), 0) as min_salary,
        ROUND(MAX(salary_max), 0) as max_salary,
        SUM(CASE WHEN salary_avg IS NOT NULL THEN 1 ELSE 0 END) as jobs_with_salary
    FROM jobs
    GROUP BY category
),
ranked AS (
    SELECT *,
        RANK() OVER (ORDER BY avg_salary DESC) as salary_rank,
        RANK() OVER (ORDER BY total_jobs DESC) as demand_rank
    FROM category_stats
    WHERE jobs_with_salary >= 50
)
SELECT 
    category,
    total_jobs,
    demand_rank,
    avg_salary,
    salary_rank,
    min_salary,
    max_salary
FROM ranked
ORDER BY avg_salary DESC
"""

result1 = pd.read_sql(query1, conn)
print(result1.to_string(index=False))

                              category  total_jobs  demand_rank  avg_salary  salary_rank  min_salary  max_salary
              CEO & General Management         120           25     22741.0            1      2000.0    500000.0
                Real Estate & Property         705           17      6992.0            2       800.0    100000.0
Information & Communication Technology        8675            3      6422.0            3         0.0    180000.0
                 Consulting & Strategy         315           22      5844.0            4       700.0     20000.0
          Banking & Financial Services        2641            9      5784.0            5         0.0    150000.0
                                 Sales        5715            5      5708.0            6         0.0    300000.0
                  Science & Technology         551           19      5617.0            7       500.0    160000.0
                          Construction        1621           11      5425.0            8       7

### Finding
- **ICT ranks #3 in salary** (RM 6,422/month) with 8,675 openings; best sector for data professionals
- **Accounting has the MOST jobs** (#1 demand, 11,308 openings) but only #13 in salary, high competition, average pay
- **CEO & General Management** tops salary at RM 22,741 but only 120 openings- very niche
- **Key insight:** High demand does NOT mean high salary in Malaysia's job market

## 3. Query 2:- Salary Ranking by Location
Filtering to locations with **200+ job postings** for statistical reliability.  
Ranking by average salary to identify the highest paying cities in Malaysia.

This query answers: Which Malaysian city should a data professional target for highest pay?

In [8]:
query2 = """
WITH location_stats AS (
    SELECT
        location,
        COUNT(*) as total_jobs,
        ROUND(AVG(salary_avg), 0) as avg_salary,
        SUM(CASE WHEN job_type = 'Full time' THEN 1 ELSE 0 END) as fulltime_jobs
    FROM jobs
    GROUP BY location
    HAVING total_jobs >= 200
)
SELECT *,
    RANK() OVER (ORDER BY avg_salary DESC) as salary_rank
FROM location_stats
ORDER BY avg_salary DESC
LIMIT 15
"""

result2 = pd.read_sql(query2, conn)
print(result2.to_string(index=False))

                              location  total_jobs  avg_salary  fulltime_jobs  salary_rank
                                Penang        1930      7216.0           1832            1
              Kuala Lumpur City Centre        2112      6217.0           2032            2
                          Kuala Lumpur       15390      5468.0          13646            3
                         Bangsar South         514      5268.0            489            4
               Petaling Jaya, Selangor         519      5253.0            474            5
Kuala Lumpur City Centre, Kuala Lumpur         438      5217.0            410            6
                                Sepang         865      5149.0            702            7
                               Bangsar         254      5091.0            225            8
                             Putrajaya         242      5053.0            195            9
                      Kuantan District         305      5020.0            222           10

### Key Finding - The Penang Surprise
- **Penang averages RM 7,216/month — 32% higher than Kuala Lumpur (RM 5,468)**
- This is non-obvious since KL has 15,390 job postings vs Penang's 1,930
- Higher average likely driven by Penang's strong manufacturing and tech sector concentration
- **Practical implication:** Job seekers targeting salary over volume should consider Penang over KL

## 4. Query 3:- ICT Roles Salary Breakdown
Drilling into the **ICT sector specifically** to identify which tech roles pay the most  
and have the most openings in Malaysia.

This query answers: Which ICT role offers the best balance of salary and job availability?

In [9]:
# Query 3 - ICT jobs specifically, salary by role
query3 = """
WITH ict_roles AS (
    SELECT
        subcategory,
        COUNT(*) as total_jobs,
        ROUND(AVG(salary_avg), 0) as avg_salary
    FROM jobs
    WHERE category = 'Information & Communication Technology'
    AND salary_avg IS NOT NULL
    GROUP BY subcategory
    HAVING total_jobs >= 20
)
SELECT *,
    RANK() OVER (ORDER BY avg_salary DESC) as salary_rank
FROM ict_roles
ORDER BY avg_salary DESC
"""

result3 = pd.read_sql(query3, conn)
print(result3.to_string(index=False))

                          subcategory  total_jobs  avg_salary  salary_rank
                Engineering - Network          20     13358.0            1
       Programme & Project Management         124      8391.0            2
                   Sales - Pre & Post          57      7746.0            3
               Engineering - Software         390      7492.0            4
                             Security          78      7423.0            5
                          Consultants          60      7192.0            6
               Developers/Programmers         632      6747.0            7
                           Management         147      6420.0            8
Database Development & Administration          55      6334.0            9
          Testing & Quality Assurance         102      6323.0           10
     Product Management & Development          55      6166.0           11
            Business/Systems Analysts         182      6108.0           12
                         

### Finding
- **Network Engineering** tops ICT at RM 13,358/month, but only 20 openings, very niche
- **Software Engineering:** RM 7,492 with 390 openings that  is strong balance of pay and demand
- **Business/Systems Analysts:** RM 6,108 with 182 openings which is most relevant for data analyst roles
- **Help Desk & IT Support:** RM 4,313, lowest paying ICT role, avoid if salary is priority

## 5. Summary of Key Insights

| Finding | Detail |
|---------|--------|
| Average Malaysian salary | RM 4,777/month |
| Highest paying city | Penang — RM 7,216/month |
| Penang vs KL premium | 32% higher despite fewer jobs |
| Best sector for data professionals | ICT — RM 6,422/month, 8,675 openings |
| Salary transparency | 54% of employers don't list salary |
| Most jobs | Accounting — 11,308 openings |
| Full time jobs | 90% of all postings |
| Top ICT role by salary | Network Engineering — RM 13,358/month |
| Best ICT role for analysts | Business/Systems Analyst — RM 6,108, 182 openings |

